# LAB 2
**Powered by:** Hapii Denys IP-05

**Topic:** Intelligent agents

In [ ]:
Console.WriteLine(2+2);

### LAB1-PART
Generated grpah with some changes

In [49]:
using XPlot.Plotly; 
using System;

//parameters init
var nodesCount = 25;
var deleteCount = 10;
var size = (int)Math.Sqrt(nodesCount);
var nodesList = new List<int>[nodesCount];
var random = new Random();
var id = 0;
var arrX = new int[nodesCount];
var arrY = new int[nodesCount];
var graph = new List<Graph.Scatter>();

// init all nodes in rect graph
for (int y = 0; y < size; y++)
{
    for (int x = 0; x < size; x++)
    {
        nodesList[id] = new List<int>();
        if (x != size - 1) nodesList[id].Add(id + 1);
        if (y != size - 1) nodesList[id].Add(id + size);
        if (x != 0) nodesList[id].Add(id - 1);
        if (y != 0) nodesList[id].Add(id - size);
        arrX[id] = x;
        arrY[id] = y;
        id++;
    }
}
var nodes = new Graph.Scatter()
{
    x = arrX,
    y = arrY,
    mode = "markers",
    marker = new Graph.Marker() {color = "blue"},
};
graph.Add(nodes);
//

// deep first search
void DFS(int source, List<int>[] adjList, bool[] visited){
    visited[source] = true;
    for (int i = 0; i < adjList[source].Count ; i++) {
        int neighbor = adjList[source][i];
        if (visited[neighbor] == false){
            DFS(neighbor, adjList, visited);
        }
    }
}
//

// check if graph still connected (agent could reach any node)
bool IsConnected() {
    var adjList = nodesList;
    var visited = new bool[nodesCount];

    DFS(0, adjList, visited);

    int count = 0;
    for (int i = 0; i < visited.Length ; i++) 
    {
        if (visited[i])
        count++;
    }

    return (nodesCount == count) ? true : false;
}
//

// remove random roads
while (deleteCount > 0)
{
    var num = random.Next(0, nodesList.Length);
    var neighborNode = -1;
    var neighborX = random.Next(-1, 1);
    if (neighborX != 0) neighborNode = num + neighborX;
    else
    {    
        var neighborY = random.Next(-1, 1);
        if (neighborY == 1) neighborY = size;
        if (neighborY == -1) neighborY = -size;
        neighborNode = num - neighborY;
    }

    if (neighborNode < 0 || neighborNode > nodesCount) continue;

    if (nodesList[num].Contains(neighborNode))
    {
        nodesList[num].Remove(neighborNode);
        nodesList[neighborNode].Remove(num);
        if (IsConnected())
        {
            deleteCount--;
        } else 
        {
            nodesList[num].Add(neighborNode);
            nodesList[neighborNode].Add(num);
        }
    }
    
}
//

// create roads (to display it correctly at graph/chart)
var counterX = 0;
var counterY = 0;
foreach(List<int> node in nodesList)
{
    if (counterX == size)
    {
        counterX = 0;
        counterY++;
    }
    Console.Write("Node " + counterX + "-" + counterY + ": ");
    foreach (int edge in node)
    {
        Console.Write(edge + ", ");
        graph.Add(new Graph.Scatter()
            {
                x = new int[] {counterX, edge % size},
                y = new int[] {counterY, (int)Math.Floor((float)edge / size)},
                mode = "lines",
                line = new Graph.Line() {color = "blue"},
            });
    }
    Console.WriteLine();
    counterX++;
}
//

// create chart
var chart = Chart.Plot(graph);
chart.WithLegend(false);
chart.Width = 500;
chart.Height = 500;
//

//show chart
display(chart);

Node 0-0: 5, 
Node 1-0: 2, 
Node 2-0: 3, 7, 1, 
Node 3-0: 4, 8, 2, 
Node 4-0: 9, 3, 
Node 0-1: 6, 0, 
Node 1-1: 11, 5, 
Node 2-1: 8, 12, 2, 
Node 3-1: 13, 7, 3, 
Node 4-1: 14, 4, 
Node 0-2: 11, 15, 
Node 1-2: 12, 10, 6, 
Node 2-2: 13, 17, 11, 7, 
Node 3-2: 14, 18, 12, 8, 
Node 4-2: 13, 9, 
Node 0-3: 16, 20, 10, 
Node 1-3: 21, 15, 
Node 2-3: 22, 12, 
Node 3-3: 23, 13, 19, 
Node 4-3: 18, 
Node 0-4: 21, 15, 
Node 1-4: 22, 20, 16, 
Node 2-4: 23, 21, 17, 
Node 3-4: 24, 22, 18, 
Node 4-4: 23, 


### Lab2 PART

Based on the previous laboratory work, 
develop a rational intelligent agent-car 
that can remember the traveled path 
and has a coefficient of efficiency.

The goal of the agent is to reach the target point on the map, 
starting from the starting point.





#### Agent `start` and `end` coords 

In [50]:
// ALL COORDS SHOULD BE IN RANGE: 0 <= `coord` < size
var agentStartX = 0;
var agentStartY = 0;

var agentEndX = 4;
var agentEndY = 4;

#### Agent moving

In [51]:
var agentCurrentX = agentStartX;
var agentCurrentY = agentStartY;
var agentPrevX = agentCurrentX;
var agentPrevY = agentCurrentY;
var currentNodeI = (agentCurrentX % size) + (agentCurrentY * size); // transform 2d into 1d
var goalNodeI = (agentEndX % size) + (agentEndY * size); // transform 2d into 1d
var roadsMemory = new Stack<int>(); // stack with remembering `correct` path of agent
var visited = new List<int>(); // list of all visited nodes
var graphWithRoads = graph; // make copy for new graph

// visit start point
Console.Write("(" + agentCurrentX + "," + agentCurrentY + ") > ");
roadsMemory.Push(currentNodeI);
visited.Add(currentNodeI);
//

// loop until we not reach end point
while (currentNodeI != goalNodeI) 
{
    var nextNode = -1;
    var heuristicCurr = Math.Abs(agentCurrentX - agentEndX) + 
                Math.Abs(agentCurrentY - agentEndY);
    foreach(int edge in nodesList[currentNodeI])
    {
        if(!visited.Contains(edge))
        {
            if (nextNode == -1) nextNode = edge;
            var heuristicNext = Math.Abs(edge % size - agentEndX) + 
                Math.Abs((int)Math.Floor((float)edge / size) - agentEndY);
            if (heuristicNext < heuristicCurr)
            {
                nextNode = edge;
            }
        }
    }
    
    if (nextNode == -1)
        roadsMemory.Pop(); // move back
    else
    {
      roadsMemory.Push(nextNode); // add node to path
      visited.Add(nextNode); // mark as visited
    }
    
    currentNodeI = roadsMemory.Peek();
    agentPrevX = agentCurrentX;
    agentPrevY = agentCurrentY;
    agentCurrentX = currentNodeI % size;
    agentCurrentY = (int)Math.Floor((float)currentNodeI / size);
    graphWithRoads.Add(new Graph.Scatter()
        {
            x = new int[] {agentPrevX, agentCurrentX},
            y = new int[] {agentPrevY, agentCurrentY},
            mode = "lines",
            line = new Graph.Line() {color = "yellow"},
        });  
    Console.Write("(" + agentCurrentX + "," + agentCurrentY + ") > ");
}
//

// graph of path to chart
agentPrevX = agentEndX;
agentPrevY = agentEndY;
foreach(int node in roadsMemory)
{
    agentCurrentX = node % size;
    agentCurrentY = (int)Math.Floor((float)node / size);
    graphWithRoads.Add(new Graph.Scatter()
        {
            x = new int[] {agentPrevX, agentCurrentX},
            y = new int[] {agentPrevY, agentCurrentY},
            mode = "lines",
            line = new Graph.Line() {color = "red"},
        }); 
    agentPrevX = agentCurrentX;
    agentPrevY = agentCurrentY;
}
//

var chartWithRoads = Chart.Plot(graphWithRoads);
chartWithRoads.WithLegend(false);
chartWithRoads.Width = 500;
chartWithRoads.Height = 500;
display(chartWithRoads);

(0,0) > (0,1) > (1,1) > (1,2) > (2,2) > (2,3) > (2,4) > (3,4) > (4,4) > 